In [5]:
import bw2data, bw2calc, bw2io
from edges import EdgeLCIA, CostLCIA, get_available_methods
from bw2analyzer.contribution import ContributionAnalysis
#bw2data.projects.set_current("EdgeLCC-Test-bw25")
bw2data.projects.set_current("bw25_ei310")
act = bw2data.Database("ecoinvent-3.10.1-cutoff").random()
#act = [a for a in bw2data.Database("lcc-test-db") if a["name"] == "Use of chair"][0]
act

'hydroquinone production' (kilogram, RER, None)

In [6]:
#method = ('GeoPolRisk', '2024')
method = ('LCC 1.0', '2023')
LCA = CostLCIA(
    demand={act: 1},
    method=method,
    #filepath="lcia_cost_example.json"
    #use_distributions=True,
    #iterations=1000
)

In [7]:
LCA.lci()
LCA.map_exchanges()
#LCA.map_aggregate_locations()
#LCA.map_contained_locations()
#LCA.map_remaining_locations_to_global()

/Users/romain/micromamba/envs/edges/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 4.83e+13)
  warnings.warn(msg, UmfpackWarning)


Mapping 167972 exchanges...


Mapping exchanges: 100%|████████████████████| 6400/6400 [03:12<00:00, 33.32it/s]

Processed edges: 6615


In [8]:
LCA.build_price_vector()
#LCA.evaluate_cfs()

Build price vector


In [9]:
LCA.infer_missing_costs()

Inferring missing costs in the technosphere matrix...
Price vector stats:
  Min: 0.0
  Max: 61131.509
  NaNs: 0
  Zeros: 22674
  Anchored prices: 1614 / 23523
A_ub diagnostics:
  Max abs value: 1000.0
  Non-zeros per row (min/max): 2 / 411
  Total constraints (rows): 21909
  Total variables (columns): 23523
Linear programming failed.
Status: 4
Message: (HiGHS Status 0: Not Set)
Number of variables: 23523
Number of constraints: 21909


RuntimeError: Linear program failed to find a consistent price vector.

In [11]:
LCA.evaluate_cfs()

In [12]:
LCA.lcia()

In [13]:
LCA.score

-0.2770328465052875

In [14]:
df = LCA.generate_cf_table()

In [15]:
df.to_excel("exchanges_with_costs.xlsx")

In [4]:
len(LCA.processed_technosphere_edges)

908

In [5]:
len(LCA.unprocessed_technosphere_edges)

167077

In [6]:
LCA.unprocessed_technosphere_edges[0]

(13884, 15683)

In [4]:
# LCA.technosphere_flow_matrix.todense()

Build price vector


In [10]:
type(LCA.price_vector)

numpy.ndarray

In [17]:
activities_missing, activities_non_missing = [], []
for x, p in enumerate(LCA.price_vector.tolist()):
    if p == 0:
        activities_missing.append(x)
    else:
        activities_non_missing.append(x)

In [26]:
d = []
for x in activities_non_missing:
    act = bw2data.get_activity(LCA.reversed_activity[x])
    classification = None
    if "classifications" in act:
        for c in act["classifications"]:
            if c[0].lower() == "cpc":
                classification = c[1].split(":")[0].strip()
    d.append(
        [act["name"], act["reference product"], act["location"], classification]
    )
import pandas as pd
pd.DataFrame(d, columns=["name", "reference product", "location", "CPC code"]).to_excel("identified.xlsx")

In [27]:
d = []
for x in activities_missing:
    act = bw2data.get_activity(LCA.reversed_activity[x])
    classification = None
    if "classifications" in act:
        for c in act["classifications"]:
            if c[0].lower() == "cpc":
                classification = c[1].split(":")[0].strip()
    d.append(
        [act["name"], act["reference product"], act["location"], classification]
    )
import pandas as pd
pd.DataFrame(d, columns=["name", "reference product", "location", "CPC code"]).to_excel("non_identified.xlsx")

In [19]:
len(activities_missing)

23350

In [20]:
LCA.reversed_activity

{0: 183941461492965376,
 1: 183941461492965377,
 2: 183941461497159680,
 3: 183941461497159681,
 4: 183941461497159682,
 5: 183941461509742592,
 6: 183941461509742593,
 7: 183941461509742594,
 8: 183941461513936896,
 9: 183941461513936897,
 10: 183941461513936898,
 11: 183941461513936899,
 12: 183941461513936900,
 13: 183941461513936901,
 14: 183941461526519808,
 15: 183941461543297024,
 16: 183941461555879936,
 17: 183941461555879937,
 18: 183941461560074240,
 19: 183941461568462848,
 20: 183941461568462849,
 21: 183941461568462850,
 22: 183941461568462851,
 23: 183941461568462852,
 24: 183941461568462853,
 25: 183941461585240064,
 26: 183941461585240065,
 27: 183941461589434368,
 28: 183941461589434369,
 29: 183941461589434370,
 30: 183941461602017280,
 31: 183941461602017281,
 32: 183941461602017282,
 33: 183941461606211584,
 34: 183941461606211585,
 35: 183941461606211586,
 36: 183941461606211587,
 37: 183941461614600192,
 38: 183941461618794496,
 39: 183941461618794497,
 40: 18394

In [6]:
LCA.price_vector.sum()

145826.247

In [10]:
LCA.price_vector

array([ 5. ,  1.9,  1. , 25. , 11.5, -2. ])

In [8]:
LCA.evaluate_cfs()

In [9]:
df = LCA.generate_cf_table()

In [10]:
df

,supplier name,supplier reference product,supplier location,consumer name,consumer reference product,consumer location,amount,CF,impact
0,spodumene production,spodumene,AU,spodumene production,spodumene,AU,1.095705e-09,3.592,3.935771e-09
1,spodumene production,spodumene,AU,market for spodumene,spodumene,GLO,-1.095705e-09,3.592,-3.935771e-09
2,market for hard coal,hard coal,AU,primary zinc production from concentrate,cobalt,RoW,-1.659007e-11,0.253,-4.197288e-12
3,market for hard coal,hard coal,AU,cobalt production,"copper, anode",GLO,-1.391981e-10,0.253,-3.521712e-11
4,market for hard coal,hard coal,AU,"hard coal, import from AU",hard coal,ZA,-6.075272e-07,0.253,-1.537044e-07
...,...,...,...,...,...,...,...,...,...
903,"photovoltaic panel production, a-Si","photovoltaic panel, a-Si",US,"market for photovoltaic panel, a-Si","photovoltaic panel, a-Si",GLO,-2.013602e-13,1258.081,-2.533274e-10
904,"photovoltaic laminate production, CdTe","photovoltaic laminate, CdTe",US,"photovoltaic laminate production, CdTe","photovoltaic laminate, CdTe",US,4.043167e-13,1258.081,5.086632e-10
905,"photovoltaic laminate production, CdTe","photovoltaic laminate, CdTe",US,"market for photovoltaic laminate, CdTe","photovoltaic laminate, CdTe",GLO,-4.043167e-13,1258.081,-5.086632e-10
906,"photovoltaic laminate production, a-Si","photovoltaic laminate, a-Si",US,"photovoltaic laminate production, a-Si","photovoltaic laminate, a-Si",US,2.013602e-13,1258.081,2.533274e-10


In [12]:
LCA.characterization_matrix.todense()

matrix([[ 5.,  5.,  0.,  5.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  1.,  1.,  0.,  0.,  0.],
        [ 0.,  0.,  0., 25., 25.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0., -2., -2.]])

In [13]:
LCA.post_process_characterization_matrix()

In [14]:
LCA.characterization_matrix.todense()

matrix([[ 5. ,  5. ,  0. ,  5. ,  0. ,  0. ],
        [ 0. ,  1.9,  0. ,  1.9,  0. ,  0. ],
        [ 0. ,  1. ,  1. ,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. , 25. , 25. ,  0. ],
        [ 0. ,  0. ,  0. ,  0. , 11.5,  0. ],
        [ 0. ,  0. ,  0. ,  0. , -2. , -2. ]])

In [15]:
LCA.technosphere_flow_matrix.todense()

matrix([[ 0.25, -0.05,  0.  , -0.2 ,  0.  ,  0.  ],
        [ 0.  ,  0.5 ,  0.  , -0.5 ,  0.  ,  0.  ],
        [ 0.  , -0.7 ,  0.7 ,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  0.5 , -0.5 ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  0.  ,  1.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  0.  , -0.5 ,  0.5 ]])

In [16]:
LCA.lcia()

In [17]:
LCA.characterized_inventory.todense()

matrix([[  1.25,  -0.25,   0.  ,  -1.  ,   0.  ,   0.  ],
        [  0.  ,   0.95,   0.  ,  -0.95,   0.  ,   0.  ],
        [  0.  ,  -0.7 ,   0.7 ,   0.  ,   0.  ,   0.  ],
        [  0.  ,   0.  ,   0.  ,  12.5 , -12.5 ,   0.  ],
        [  0.  ,   0.  ,   0.  ,   0.  ,  11.5 ,   0.  ],
        [  0.  ,   0.  ,   0.  ,   0.  ,   1.  ,  -1.  ]])

In [18]:
LCA.characterized_inventory.sum()

11.5